In [ ]:
#!pip install mediapipe
import cv2
import mediapipe as mp
import numpy as np

class FaceModelGenerator:
    def __init__(self):
        self.mp_face_mesh = mp.solutions.face_mesh
        self.face_mesh = self.mp_face_mesh.FaceMesh(
            static_image_mode=True,
            max_num_faces=1,
            min_detection_confidence=0.5
        )
        # Load the face mesh tesselation
        self.mp_face_mesh_tesselation = mp.solutions.face_mesh_connections


    def generate_obj_from_image(self, image_path, output_path='/content/drive/MyDrive/OutputPaper2/face_model.obj'):
        """Generate a 3D face model OBJ file from an image"""
        # Read image
        image = cv2.imread(image_path)
        if image is None:
            raise ValueError("Could not read image file")

        # Convert to RGB for MediaPipe
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Detect face landmarks
        results = self.face_mesh.process(image_rgb)

        if not results.multi_face_landmarks:
            raise ValueError("No face detected in image")

        # Get face landmarks
        face_landmarks = results.multi_face_landmarks[0]

        # Convert landmarks to 3D points
        vertices = []
        for landmark in face_landmarks.landmark:
            vertices.append([
                landmark.x * image.shape[1],
                -landmark.y * image.shape[0],  # Flip Y for standard 3D coordinate system
                landmark.z * image.shape[1]  # Scale Z similarly to X
            ])
        vertices = np.array(vertices)

        # Define faces using MediaPipe's face mesh tesselation
        # Get the connections from FACEMESH_TESSELATION
        connections = self.mp_face_mesh_tesselation.FACEMESH_TESSELATION
         # Extract the faces (triangles) from the connections
        faces = []
        for connection in connections:
            # Assuming each connection represents an edge of a triangle
            # We need to find the third vertex to form the triangle
            # This is a simplification and might not be completely accurate
            # You might need to adjust this logic based on the actual data in FACEMESH_TESSELATION
            v1, v2 = connection
            # Find a third vertex connected to both v1 and v2 (this is a heuristic)
            for other_connection in connections:
                if v1 in other_connection and v2 not in other_connection:
                    v3 = other_connection[0] if other_connection[1] == v1 else other_connection[1]
                    faces.append([v1, v2, v3])
                    break
                elif v2 in other_connection and v1 not in other_connection:
                    v3 = other_connection[0] if other_connection[1] == v2 else other_connection[1]
                    faces.append([v1, v2, v3])
                    break

        # Write OBJ file
        with open(output_path, 'w') as f:
            # Write vertices
            for vertex in vertices:
                f.write(f'v {vertex[0]:.6f} {vertex[1]:.6f} {vertex[2]:.6f}\n')

            # Write faces (OBJ uses 1-based indexing)
            for face in faces:
                f.write(f'f {face[0]+1} {face[1]+1} {face[2]+1}\n')

        return output_path

    def generate_with_texture(self, image_path, output_base_path='/content/drive/MyDrive/OutputPaper2/face_model'):
        """Generate a textured 3D face model with materials"""
        # Read and process image as before
        image = cv2.imread(image_path)
        if image is None:
            raise ValueError("Could not read image file")

        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = self.face_mesh.process(image_rgb)

        if not results.multi_face_landmarks:
            raise ValueError("No face detected in image")

        # Generate UV coordinates
        face_landmarks = results.multi_face_landmarks[0]
        vertices = []
        uv_coords = []

        for landmark in face_landmarks.landmark:
            vertices.append([
                landmark.x * image.shape[1],
                -landmark.y * image.shape[0],
                landmark.z * image.shape[1]
            ])
            # Generate UV coordinates (simple mapping)
            uv_coords.append([landmark.x, landmark.y])

        vertices = np.array(vertices)
       # Access face mesh tesselation
        # Get the connections from FACEMESH_TESSELATION
        connections = self.mp_face_mesh_tesselation.FACEMESH_TESSELATION

        # Extract the faces (triangles) from the connections
        faces = []
        for connection in connections:
            # Assuming each connection represents an edge of a triangle
            # We need to find the third vertex to form the triangle
            # This is a simplification and might not be completely accurate
            # You might need to adjust this logic based on the actual data in FACEMESH_TESSELATION
            v1, v2 = connection
            # Find a third vertex connected to both v1 and v2 (this is a heuristic)
            for other_connection in connections:
                if v1 in other_connection and v2 not in other_connection:
                    v3 = other_connection[0] if other_connection[1] == v1 else other_connection[1]
                    faces.append([v1, v2, v3])
                    break
                elif v2 in other_connection and v1 not in other_connection:
                    v3 = other_connection[0] if other_connection[1] == v2 else other_connection[1]
                    faces.append([v1, v2, v3])
                    break

        # Write OBJ with texture coordinates
        obj_path = f'{output_base_path}.obj'
        mtl_path = f'{output_base_path}.mtl'
        texture_path = f'{output_base_path}_texture.png'

        # Save texture image
        cv2.imwrite(texture_path, image)

        # Write MTL file
        with open(mtl_path, 'w') as f:
            f.write('newmtl material0\n')
            f.write('Ka 1.000000 1.000000 1.000000\n')
            f.write('Kd 1.000000 1.000000 1.000000\n')
            f.write('Ks 0.000000 0.000000 0.000000\n')
            f.write(f'map_Kd {output_base_path}_texture.png\n')

        # Write OBJ file
        with open(obj_path, 'w') as f:
            f.write(f'mtllib {output_base_path}.mtl\n')

            # Write vertices
            for vertex in vertices:
                f.write(f'v {vertex[0]:.6f} {vertex[1]:.6f} {vertex[2]:.6f}\n')

            # Write texture coordinates
            for uv in uv_coords:
                f.write(f'vt {uv[0]:.6f} {uv[1]:.6f}\n')

            # Write faces with texture coordinates
            f.write('usemtl material0\n')
            for face in faces:
                # OBJ uses 1-based indexing
                f.write(f'f {face[0]+1}/{face[0]+1} {face[1]+1}/{face[1]+1} {face[2]+1}/{face[2]+1}\n')

        return obj_path, mtl_path, texture_path

def main():
    # Initialize generator
    generator = FaceModelGenerator()

    # Generate basic model
    try:
        obj_path = generator.generate_obj_from_image('/content/drive/MyDrive/InputPaper2/FaceModel.png')
        print(f"Basic face model generated: {obj_path}")
    except Exception as e:
        print(f"Error generating basic model: {str(e)}")

    # Generate textured model
    try:
        obj_path, mtl_path, texture_path = generator.generate_with_texture('/content/drive/MyDrive/InputPaper2/FaceModel.png')
        print(f"Textured face model generated:")
        print(f"OBJ file: {obj_path}")
        print(f"MTL file: {mtl_path}")
        print(f"Texture file: {texture_path}")
    except Exception as e:
        print(f"Error generating textured model: {str(e)}")

if __name__ == "__main__":
    main()

Basic face model generated: /content/drive/MyDrive/OutputPaper2/face_model.obj
Textured face model generated:
OBJ file: /content/drive/MyDrive/OutputPaper2/face_model.obj
MTL file: /content/drive/MyDrive/OutputPaper2/face_model.mtl
Texture file: /content/drive/MyDrive/OutputPaper2/face_model_texture.png


In [ ]:
#!pip install pycryptodome
#!pip install trimesh
import numpy as np
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
import hashlib
import secrets
import struct
import trimesh
import scipy.spatial as spatial

class SecureFaceSteganography:
    def __init__(self):
        # ... (other code)
        self.LANDMARK_REGIONS = {
            'forehead': [(0.2, 0.8, 0.5), (0.8, 0.8, 0.5)],  # Wider bounding box
            'cheeks': [(0.1, 0.5, 0.5), (0.9, 0.5, 0.5)],  # Wider bounding box
            'jaw': [(0.2, 0.2, 0.5), (0.8, 0.2, 0.5)],  # Wider bounding box
            'nose': [(0.4, 0.6, 0.5), (0.6, 0.6, 0.5)],  # Added nose region
            'chin': [(0.5, 0.1, 0.5)]  #chin region
        }
        # Define SALT_SIZE and BLOCK_SIZE in the __init__ method
        self.SALT_SIZE = 16  # 128 bits for salt
        self.BLOCK_SIZE = 16 # 128 bits for block size

    def load_face_model(self, file_path):
        """Load 3D face model and prepare it for steganography"""
        try:
            mesh = trimesh.load(file_path)
            return {
                'vertices': np.array(mesh.vertices),
                'faces': np.array(mesh.faces),
                'normals': np.array(mesh.vertex_normals)
            }
        except Exception as e:
            raise ValueError(f"Error loading face model: {str(e)}")

    def identify_landmark_vertices(self, vertices, landmark_regions=None):
        """Identify vertices near facial landmarks for data embedding"""
        if landmark_regions is None:
            landmark_regions = self.LANDMARK_REGIONS

        landmark_vertices = []
        kdtree = spatial.KDTree(vertices)
        for region, points in landmark_regions.items():
            for point in points:
                # Find nearest vertices to landmark points
                # Increased k to 100 (or even higher if needed) to find more neighboring vertices
                distances, indices = kdtree.query(point, k=1000)  # Increased k here
                landmark_vertices.extend(indices)
        return sorted(set(landmark_vertices))  # Remove duplicates

    def compute_vertex_importance(self, vertices, faces, normals):
        """Compute vertex importance based on geometric features"""
        importance = np.zeros(len(vertices))

        # Calculate curvature (simplified)
        for i, vertex in enumerate(vertices):
            # Find connected vertices
            connected = faces[np.any(faces == i, axis=1)]
            connected = np.unique(connected.flatten())
            connected = connected[connected != i]

            # Calculate local curvature
            if len(connected) > 0:
                connected_vertices = vertices[connected]
                mean_position = np.mean(connected_vertices, axis=0)
                displacement = np.linalg.norm(vertex - mean_position)
                importance[i] = displacement

        return importance

    def generate_passphrase(self, length=32):
        """Generate a cryptographically secure random passphrase"""
        charset = "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789!@#$%^&*"
        return ''.join(secrets.choice(charset) for _ in range(length))

    def derive_key(self, passphrase, salt):
        """Derive encryption key using SHA-256 with salt"""
        key_material = hashlib.sha256()
        key_material.update(passphrase.encode('utf-8'))
        key_material.update(salt)
        return key_material.digest()

    def encrypt_message(self, message, passphrase):
        """Encrypt a message using AES-128-CBC with secure key derivation"""
        salt = secrets.token_bytes(self.SALT_SIZE)
        key = self.derive_key(passphrase, salt)
        iv = secrets.token_bytes(self.BLOCK_SIZE)

        cipher = AES.new(key, AES.MODE_CBC, iv)
        padded_data = pad(message.encode('utf-8'), self.BLOCK_SIZE)
        ciphertext = cipher.encrypt(padded_data)

        return salt + iv + ciphertext

    def decrypt_message(self, encrypted_data, passphrase):
        """Decrypt a message using AES-128-CBC"""
        salt = encrypted_data[:self.SALT_SIZE]
        iv = encrypted_data[self.SALT_SIZE:self.SALT_SIZE + self.BLOCK_SIZE]
        ciphertext = encrypted_data[self.SALT_SIZE + self.BLOCK_SIZE:]

        key = self.derive_key(passphrase, salt)
        cipher = AES.new(key, AES.MODE_CBC, iv)

        # Attempt to decrypt. Handle potential padding errors
        try:
            padded_message = cipher.decrypt(ciphertext)
            return unpad(padded_message, self.BLOCK_SIZE).decode('utf-8')
        except ValueError:
            print("Warning: Decryption failed. Possible data corruption or incorrect passphrase.")
            return None  # or raise a more specific exception


    def embed_in_face_model(self, face_model, encrypted_data):
        """Embed encrypted data in 3D face model using geometric features"""
        vertices = face_model['vertices'].copy()
        faces = face_model['faces']
        normals = face_model['normals']

        # Get landmarks and importance metrics
        landmark_vertices = self.identify_landmark_vertices(vertices)
        importance = self.compute_vertex_importance(vertices, faces, normals)

        # Convert encrypted data to bit array
        bit_array = ''.join(format(byte, '08b') for byte in encrypted_data)

        # -- Check if the message is too large --
        if len(bit_array) > len(landmark_vertices):
            raise ValueError(f"Message too large for this face model. Message bits: {len(bit_array)}, Available vertices: {len(landmark_vertices)}")

        # Embed data in vertices near landmarks
        for i, bit in enumerate(bit_array):
            vertex_idx = landmark_vertices[i]

            # Reduce scale_factor to embed less data and reduce potential corruption
            scale_factor = 0.000005 * (1 - importance[vertex_idx])  # Reduced scale_factor

            # Embed in least significant component while preserving facial features
            vertices[vertex_idx] += (float(bit) * scale_factor * normals[vertex_idx])

        face_model['vertices'] = vertices
        return face_model

    def extract_from_face_model(self, face_model, original_model, message_length):
        """Extract encrypted data from modified 3D face model"""
        modified_vertices = face_model['vertices']
        original_vertices = original_model['vertices']
        landmark_vertices = self.identify_landmark_vertices(original_vertices)

        bits = []
        for i in range(message_length * 8):  # Iterate over the expected number of bits
            if i >= len(landmark_vertices):
                print(f"Warning: Not enough landmark vertices to extract the full message. Extracted {i} bits, expected {message_length * 8} bits.")
                break  # Stop if we run out of landmark vertices
            vertex_idx = landmark_vertices[i]
            # Compare modified vertex with original
            diff = modified_vertices[vertex_idx] - original_vertices[vertex_idx]
            # Extract bit based on vertex displacement
            bit = '1' if np.dot(diff, face_model['normals'][vertex_idx]) > 0 else '0'
            bits.append(bit)

        # Convert bits back to bytes
        extracted_data = bytes(
            int(''.join(bits[i:i + 8]), 2) for i in range(0, len(bits), 8)
        )

        return extracted_data

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 704.8/704.8 kB 29.0 MB/s eta 0:00:00


In [ ]:
def main():
    # Initialize steganography system
    stego = SecureFaceSteganography()

    # Load 3D face model (example path)
    face_model = stego.load_face_model('/content/drive/MyDrive/OutputPaper2/pjanic.obj')
    original_model = {key: value.copy() for key, value in face_model.items()}

    # Generate secure passphrase
    passphrase = stego.generate_passphrase()
    print(f"Generated passphrase: {passphrase}")

    # Message to hide - Reduced length to fit within the model's capacity
    secret_message = "short"

    # Encrypt the message
    encrypted_data = stego.encrypt_message(secret_message, passphrase)

    # Embed encrypted data in face model
    modified_model = stego.embed_in_face_model(face_model, encrypted_data)

    # Extract and decrypt
    extracted_data = stego.extract_from_face_model(
        modified_model, original_model, len(encrypted_data)
    )
    decrypted_message = stego.decrypt_message(extracted_data, passphrase)

    print(f"Original message: {secret_message}")
    print(f"Decrypted message: {decrypted_message}")

    # Save modified model
    mesh = trimesh.Trimesh(
        vertices=modified_model['vertices'],
        faces=modified_model['faces']
    )
    mesh.export('/content/drive/MyDrive/OutputPaper2/modified_pjanic.obj')

if __name__ == "__main__":
    main()

Generated passphrase: l*YExDuSeIiQrInqqybl5YfVVdCQYaMo
Original message: short
Decrypted message: short


In [ ]:
#!pip install pycryptodome
#!pip install trimesh
import numpy as np
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
import hashlib
import secrets
import struct
import trimesh
import scipy.spatial as spatial

class SecureFaceSteganography:
    def __init__(self):
        # ... (other code)
        self.LANDMARK_REGIONS = {
            'forehead': [(0.2, 0.8, 0.5), (0.8, 0.8, 0.5)],  # Wider bounding box
            'cheeks': [(0.1, 0.5, 0.5), (0.9, 0.5, 0.5)],  # Wider bounding box
            'jaw': [(0.2, 0.2, 0.5), (0.8, 0.2, 0.5)],  # Wider bounding box
            'nose': [(0.4, 0.6, 0.5), (0.6, 0.6, 0.5)],  # Added nose region
            'chin': [(0.5, 0.1, 0.5)]  #chin region
        }
        # Define SALT_SIZE and BLOCK_SIZE in the __init__ method
        self.SALT_SIZE = 16  # 128 bits for salt
        self.BLOCK_SIZE = 16 # 128 bits for block size

    def load_face_model(self, file_path):
        """Load 3D face model and prepare it for steganography"""
        try:
            mesh = trimesh.load(file_path)
            return {
                'vertices': np.array(mesh.vertices),
                'faces': np.array(mesh.faces),
                'normals': np.array(mesh.vertex_normals)
            }
        except Exception as e:
            raise ValueError(f"Error loading face model: {str(e)}")

    def identify_landmark_vertices(self, vertices, landmark_regions=None):
        """Identify vertices near facial landmarks for data embedding"""
        if landmark_regions is None:
            landmark_regions = self.LANDMARK_REGIONS

        landmark_vertices = []
        kdtree = spatial.KDTree(vertices)
        for region, points in landmark_regions.items():
            for point in points:
                # Find nearest vertices to landmark points
                # Increased k to 100 (or even higher if needed) to find more neighboring vertices
                distances, indices = kdtree.query(point, k=1000)  # Increased k here
                landmark_vertices.extend(indices)
        return sorted(set(landmark_vertices))  # Remove duplicates

    def compute_vertex_importance(self, vertices, faces, normals):
        """Compute vertex importance based on geometric features"""
        importance = np.zeros(len(vertices))

        # Calculate curvature (simplified)
        for i, vertex in enumerate(vertices):
            # Find connected vertices
            connected = faces[np.any(faces == i, axis=1)]
            connected = np.unique(connected.flatten())
            connected = connected[connected != i]

            # Calculate local curvature
            if len(connected) > 0:
                connected_vertices = vertices[connected]
                mean_position = np.mean(connected_vertices, axis=0)
                displacement = np.linalg.norm(vertex - mean_position)
                importance[i] = displacement

        return importance

    def generate_passphrase(self, length=32):
        """Generate a cryptographically secure random passphrase"""
        charset = "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789!@#$%^&*"
        return ''.join(secrets.choice(charset) for _ in range(length))

    def derive_key(self, passphrase, salt):
        """Derive encryption key using SHA-256 with salt"""
        key_material = hashlib.sha256()
        key_material.update(passphrase.encode('utf-8'))
        key_material.update(salt)
        return key_material.digest()

    def encrypt_message(self, message, passphrase):
        """Encrypt a message using AES-128-CBC with secure key derivation"""
        salt = secrets.token_bytes(self.SALT_SIZE)
        key = self.derive_key(passphrase, salt)
        iv = secrets.token_bytes(self.BLOCK_SIZE)

        cipher = AES.new(key, AES.MODE_CBC, iv)
        padded_data = pad(message.encode('utf-8'), self.BLOCK_SIZE)
        ciphertext = cipher.encrypt(padded_data)

        return salt + iv + ciphertext

    def decrypt_message(self, encrypted_data, passphrase):
        """Decrypt a message using AES-128-CBC"""
        salt = encrypted_data[:self.SALT_SIZE]
        iv = encrypted_data[self.SALT_SIZE:self.SALT_SIZE + self.BLOCK_SIZE]
        ciphertext = encrypted_data[self.SALT_SIZE + self.BLOCK_SIZE:]

        key = self.derive_key(passphrase, salt)
        cipher = AES.new(key, AES.MODE_CBC, iv)

        # Attempt to decrypt. Handle potential padding errors
        try:
            padded_message = cipher.decrypt(ciphertext)
            return unpad(padded_message, self.BLOCK_SIZE).decode('utf-8')
        except ValueError:
            print("Warning: Decryption failed. Possible data corruption or incorrect passphrase.")
            return None  # or raise a more specific exception


    def embed_in_face_model(self, face_model, encrypted_data):
        """Embed encrypted data in 3D face model using geometric features"""
        vertices = face_model['vertices'].copy()
        faces = face_model['faces']
        normals = face_model['normals']

        # Get landmarks and importance metrics
        landmark_vertices = self.identify_landmark_vertices(vertices)
        importance = self.compute_vertex_importance(vertices, faces, normals)

        # Convert encrypted data to bit array
        bit_array = ''.join(format(byte, '08b') for byte in encrypted_data)

        # -- Check if the message is too large --
        if len(bit_array) > len(landmark_vertices):
            raise ValueError(f"Message too large for this face model. Message bits: {len(bit_array)}, Available vertices: {len(landmark_vertices)}")

        # Embed data in vertices near landmarks
        for i, bit in enumerate(bit_array):
            vertex_idx = landmark_vertices[i]

            # Reduce scale_factor to embed less data and reduce potential corruption
            scale_factor = 0.000005 * (1 - importance[vertex_idx])  # Reduced scale_factor

            # Embed in least significant component while preserving facial features
            vertices[vertex_idx] += (float(bit) * scale_factor * normals[vertex_idx])

        face_model['vertices'] = vertices
        return face_model

    def extract_from_face_model(self, face_model, original_model, message_length):
        """Extract encrypted data from modified 3D face model"""
        modified_vertices = face_model['vertices']
        original_vertices = original_model['vertices']
        landmark_vertices = self.identify_landmark_vertices(original_vertices)

        bits = []
        for i in range(message_length * 8):  # Iterate over the expected number of bits
            if i >= len(landmark_vertices):
                print(f"Warning: Not enough landmark vertices to extract the full message. Extracted {i} bits, expected {message_length * 8} bits.")
                break  # Stop if we run out of landmark vertices
            vertex_idx = landmark_vertices[i]
            # Compare modified vertex with original
            diff = modified_vertices[vertex_idx] - original_vertices[vertex_idx]
            # Extract bit based on vertex displacement
            bit = '1' if np.dot(diff, face_model['normals'][vertex_idx]) > 0 else '0'
            bits.append(bit)

        # Convert bits back to bytes
        extracted_data = bytes(
            int(''.join(bits[i:i + 8]), 2) for i in range(0, len(bits), 8)
        )

        return extracted_data

In [ ]:
def main():
    # Initialize steganography system
    stego = SecureFaceSteganography()

    # Load 3D face model (example path)
    face_model = stego.load_face_model('/content/drive/MyDrive/OutputPaper2/pjanic.obj')
    original_model = {key: value.copy() for key, value in face_model.items()}

    # Generate secure passphrase
    passphrase = stego.generate_passphrase()
    print(f"Generated passphrase: {passphrase}")

    # Message to hide - Reduced length to fit within the model's capacity
    secret_message = "short Message"

    # Encrypt the message
    encrypted_data = stego.encrypt_message(secret_message, passphrase)

    # Embed encrypted data in face model
    modified_model = stego.embed_in_face_model(face_model, encrypted_data)

    # Extract and decrypt
    extracted_data = stego.extract_from_face_model(
        modified_model, original_model, len(encrypted_data)
    )
    decrypted_message = stego.decrypt_message(extracted_data, passphrase)

    print(f"Original message: {secret_message}")
    print(f"Decrypted message: {decrypted_message}")

    # Save modified model
    mesh = trimesh.Trimesh(
        vertices=modified_model['vertices'],
        faces=modified_model['faces']
    )
    mesh.export('/content/drive/MyDrive/OutputPaper2/modified_pjanic.obj')

if __name__ == "__main__":
    main()

Generated passphrase: TFWlxnNM*pqqCjLZgu^&6C$4YcYWGMk8
Original message: short Message
Decrypted message: short Message


In [ ]:
import trimesh
from scipy.spatial.distance import directed_hausdorff

# Load the models
original_mesh = trimesh.load('/content/drive/MyDrive/OutputPaper2/modified_pjanic.obj')
stego_mesh = trimesh.load('/content/drive/MyDrive/OutputPaper2/pjanic.obj')

# Calculate surface area
original_surface_area = original_mesh.area
stego_surface_area = stego_mesh.area

# Calculate volume
original_volume = original_mesh.volume
stego_volume = stego_mesh.volume

# Calculate Hausdorff distance
hausdorff_distance = directed_hausdorff(original_mesh.vertices, stego_mesh.vertices)[0]

# Print the results
print("Original Surface Area:", original_surface_area)
print("Stego Surface Area:", stego_surface_area)
print("Original Volume:", original_volume)
print("Stego Volume:", stego_volume)
print("Hausdorff Distance:", hausdorff_distance)

Original Surface Area: 58.79708420156814
Stego Surface Area: 58.79708350578467
Original Volume: 28.31652861618301
Stego Volume: 28.316527674660872
Hausdorff Distance: 5.004997502504824e-06


In [ ]:
#!pip install 'pyglet<2'
#!apt-get install -y libglu1-mesa-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libgl-dev libglu1-mesa libglx-dev
The following NEW packages will be installed:
  libgl-dev libglu1-mesa libglu1-mesa-dev libglx-dev
0 upgraded, 4 newly installed, 0 to remove and 49 not upgraded.
Need to get 492 kB of archives.
After this operation, 2,830 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libglx-dev amd64 1.4.0-1 [14.1 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libgl-dev amd64 1.4.0-1 [101 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 libglu1-mesa amd64 9.0.2-1 [145 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/main amd64 libglu1-mesa-dev amd64 9.0.2-1 [231 kB]
Fetched 492 kB in 0s (4,259 kB/s)
Selecting previously unselected package libglx-dev:amd64.
(Reading database ... 123632 files and directories currently installed.)
Preparing to unp

In [ ]:
#!pip install pyglet
#!pip install pyglet<2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
import trimesh
from scipy.stats import wasserstein_distance
from skimage.metrics import structural_similarity
import math
from scipy.spatial import KDTree
import os

class Stego3DEvaluator:
    def __init__(self):
        """Initialize the 3D steganography evaluator"""
        self.metrics = {}

    def preprocess_models(self, original_mesh, stego_mesh):
        """Preprocess and align models to ensure compatibility"""
        # Center both meshes
        original_mesh.vertices -= original_mesh.vertices.mean(axis=0)
        stego_mesh.vertices -= stego_mesh.vertices.mean(axis=0)

        # Scale to unit cube
        original_scale = np.max(np.abs(original_mesh.vertices))
        stego_scale = np.max(np.abs(stego_mesh.vertices))

        original_mesh.vertices /= original_scale
        stego_mesh.vertices /= stego_scale

        return original_mesh, stego_mesh

    def align_vertices(self, original_vertices, stego_vertices):
        """Align vertices between models using nearest neighbor matching"""
        # Build KD-tree for faster nearest neighbor search
        tree = KDTree(original_vertices)

        # Find nearest neighbors for each stego vertex
        distances, indices = tree.query(stego_vertices)

        # Create aligned vertex arrays
        aligned_original = original_vertices[indices]
        aligned_stego = stego_vertices

        return aligned_original, aligned_stego, distances

    def load_models(self, original_path, stego_path, align=True):
        """Load and preprocess original and stego 3D models"""
        try:
            # Load meshes
            original_mesh = trimesh.load(original_path)
            stego_mesh = trimesh.load(stego_path)

            print(f"Original vertices: {len(original_mesh.vertices)}")
            print(f"Stego vertices: {len(stego_mesh.vertices)}")

            # Preprocess meshes
            original_mesh, stego_mesh = self.preprocess_models(original_mesh, stego_mesh)

            if align:
                # Align vertices
                self.original_vertices, self.stego_vertices, distances = self.align_vertices(
                    original_mesh.vertices,
                    stego_mesh.vertices
                )

                # Print alignment statistics
                print(f"\nAlignment Statistics:")
                print(f"Mean distance: {np.mean(distances):.6f}")
                print(f"Max distance: {np.max(distances):.6f}")
                print(f"Aligned vertices: {len(self.original_vertices)}")
            else:
                if len(original_mesh.vertices) != len(stego_mesh.vertices):
                    raise ValueError("Models have different number of vertices and alignment is disabled")
                self.original_vertices = original_mesh.vertices
                self.stego_vertices = stego_mesh.vertices

            self.original_mesh = original_mesh
            self.stego_mesh = stego_mesh

            return True

        except Exception as e:
            print(f"Error loading models: {str(e)}")
            return False

    def calculate_vertex_error_map(self):
        """Calculate and visualize vertex-wise errors"""
        errors = np.linalg.norm(self.original_vertices - self.stego_vertices, axis=1)
        return errors



    def calculate_psnr(self):
        """Calculate Peak Signal-to-Noise Ratio"""
        mse = np.mean((self.original_vertices - self.stego_vertices) ** 2)
        if mse == 0:
            return float('inf')

        max_val = np.max(self.original_vertices) - np.min(self.original_vertices)
        psnr = 20 * math.log10(max_val / math.sqrt(mse))
        self.metrics['PSNR'] = psnr
        return psnr

    def calculate_ssim(self):
        """Calculate Structural Similarity Index"""
        orig_reshaped = self.original_vertices.reshape(-1, 3)
        stego_reshaped = self.stego_vertices.reshape(-1, 3)

        ssim_scores = []
        for i in range(3):
            score = structural_similarity(
                orig_reshaped[:, i],
                stego_reshaped[:, i],
                data_range=np.max(orig_reshaped[:, i]) - np.min(orig_reshaped[:, i])
            )
            ssim_scores.append(score)

        ssim = np.mean(ssim_scores)
        self.metrics['SSIM'] = ssim
        return ssim

    def calculate_mse(self):
        """Calculate Mean Squared Error"""
        mse = mean_squared_error(self.original_vertices, self.stego_vertices)
        self.metrics['MSE'] = mse
        return mse

    def calculate_rmse(self):
        """Calculate Root Mean Squared Error"""
        rmse = np.sqrt(self.calculate_mse())
        self.metrics['RMSE'] = rmse
        return rmse

    def calculate_ber(self, threshold=1e-6):
        """Calculate Bit Error Rate"""
        differences = np.abs(self.original_vertices - self.stego_vertices)
        binary_orig = (self.original_vertices > threshold).astype(int)
        binary_stego = (self.stego_vertices > threshold).astype(int)

        total_bits = np.prod(binary_orig.shape)
        error_bits = np.sum(binary_orig != binary_stego)

        ber = error_bits / total_bits
        self.metrics['BER'] = ber
        return ber

    def calculate_hausdorff_distance(self):
        """Calculate Hausdorff distance between original and stego models"""
        def directed_hausdorff(source, target):
            tree = KDTree(target)
            distances, _ = tree.query(source)
            return np.max(distances)

        forward = directed_hausdorff(self.original_vertices, self.stego_vertices)
        backward = directed_hausdorff(self.stego_vertices, self.original_vertices)

        hausdorff = max(forward, backward)
        self.metrics['Hausdorff'] = hausdorff
        return hausdorff

    def calculate_histogram_similarity(self, bins=50):
        """Calculate histogram similarity using Wasserstein distance"""
        distances = []

        for i in range(3):
            hist_orig, _ = np.histogram(self.original_vertices[:, i], bins=bins, density=True)
            hist_stego, _ = np.histogram(self.stego_vertices[:, i], bins=bins, density=True)

            distance = wasserstein_distance(hist_orig, hist_stego)
            distances.append(distance)

        hist_similarity = 1 / (1 + np.mean(distances))
        self.metrics['Histogram_Similarity'] = hist_similarity
        return hist_similarity

    def plot_histograms(self, save_path=None):
        """Plot histograms of original and stego models"""
        fig, axes = plt.subplots(1, 3, figsize=(15, 5))
        coords = ['X', 'Y', 'Z']

        for i, (ax, coord) in enumerate(zip(axes, coords)):
            ax.hist(self.original_vertices[:, i], bins=50, alpha=0.5, label='Original', density=True)
            ax.hist(self.stego_vertices[:, i], bins=50, alpha=0.5, label='Stego', density=True)
            ax.set_title(f'{coord} Coordinate Distribution')
            ax.set_xlabel(f'{coord} Value')
            ax.set_ylabel('Density')
            ax.legend()

        plt.tight_layout()
        if save_path:
            plt.savefig(save_path)
            plt.close()
        else:
            plt.show()

    def plot_error_distribution(self, save_path=None):
        """Plot the distribution of geometric errors"""
        errors = np.linalg.norm(self.original_vertices - self.stego_vertices, axis=1)

        plt.figure(figsize=(10, 6))
        plt.hist(errors, bins=50, density=True)
        plt.title('Geometric Error Distribution')
        plt.xlabel('Error Magnitude')
        plt.ylabel('Density')

        if save_path:
            plt.savefig(save_path)
            plt.close()
        else:
            plt.show()

    def evaluate_all(self, plot=True, save_plots=False):
        """Calculate all metrics and optionally generate plots"""
        metrics = {
            'PSNR': self.calculate_psnr(),
            'SSIM': self.calculate_ssim(),
            'MSE': self.calculate_mse(),
            'RMSE': self.calculate_rmse(),
            'BER': self.calculate_ber(),
            'Hausdorff': self.calculate_hausdorff_distance(),
            'Histogram_Similarity': self.calculate_histogram_similarity()
        }

        if plot:
            self.plot_histograms(save_path='histograms.png' if save_plots else None)
            self.plot_error_distribution(save_path='error_distribution.png' if save_plots else None)
            #self.visualize_error_map(save_path='error_map.png' if save_plots else None)

        return metrics

def main():
    # Initialize evaluator
    evaluator = Stego3DEvaluator()

    # Load and align models
    if not evaluator.load_models('/content/drive/MyDrive/OutputPaper2/modified_pjanic.obj', '/content/drive/MyDrive/OutputPaper2/pjanic.obj', align=True):
        return

    # Perform evaluation
    metrics = evaluator.evaluate_all(plot=True, save_plots=True)

    # Print results
    print("\nEvaluation Results:")
    for metric, value in metrics.items():
        print(f"{metric}: {value:.6f}")

if __name__ == "__main__":
    main()

Original vertices: 215999
Stego vertices: 220141

Alignment Statistics:
Mean distance: 0.000911
Max distance: 0.000959
Aligned vertices: 220141

Evaluation Results:
PSNR: 69.809654
SSIM: 0.999811
MSE: 0.000000
RMSE: 0.000526
BER: 0.000486
Hausdorff: 0.000959
Histogram_Similarity: 0.999544


In [ ]:
#!pip install pyglet
#!pip install pyglet<2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
import trimesh
from scipy.stats import wasserstein_distance
from skimage.metrics import structural_similarity
import math
from scipy.spatial import KDTree
import os

class Stego3DEvaluator:
    def __init__(self):
        """Initialize the 3D steganography evaluator"""
        self.metrics = {}

    def preprocess_models(self, original_mesh, stego_mesh):
        """Preprocess and align models to ensure compatibility"""
        # Center both meshes
        original_mesh.vertices -= original_mesh.vertices.mean(axis=0)
        stego_mesh.vertices -= stego_mesh.vertices.mean(axis=0)

        # Scale to unit cube
        original_scale = np.max(np.abs(original_mesh.vertices))
        stego_scale = np.max(np.abs(stego_mesh.vertices))

        original_mesh.vertices /= original_scale
        stego_mesh.vertices /= stego_scale

        return original_mesh, stego_mesh

    def align_vertices(self, original_vertices, stego_vertices):
        """Align vertices between models using nearest neighbor matching"""
        # Build KD-tree for faster nearest neighbor search
        tree = KDTree(original_vertices)

        # Find nearest neighbors for each stego vertex
        distances, indices = tree.query(stego_vertices)

        # Create aligned vertex arrays
        aligned_original = original_vertices[indices]
        aligned_stego = stego_vertices

        return aligned_original, aligned_stego, distances

    def load_models(self, original_path, stego_path, align=True):
        """Load and preprocess original and stego 3D models"""
        try:
            # Load meshes
            original_mesh = trimesh.load(original_path)
            stego_mesh = trimesh.load(stego_path)

            print(f"Original vertices: {len(original_mesh.vertices)}")
            print(f"Stego vertices: {len(stego_mesh.vertices)}")

            # Preprocess meshes
            original_mesh, stego_mesh = self.preprocess_models(original_mesh, stego_mesh)

            if align:
                # Align vertices
                self.original_vertices, self.stego_vertices, distances = self.align_vertices(
                    original_mesh.vertices,
                    stego_mesh.vertices
                )

                # Print alignment statistics
                print(f"\nAlignment Statistics:")
                print(f"Mean distance: {np.mean(distances):.6f}")
                print(f"Max distance: {np.max(distances):.6f}")
                print(f"Aligned vertices: {len(self.original_vertices)}")
            else:
                if len(original_mesh.vertices) != len(stego_mesh.vertices):
                    raise ValueError("Models have different number of vertices and alignment is disabled")
                self.original_vertices = original_mesh.vertices
                self.stego_vertices = stego_mesh.vertices

            self.original_mesh = original_mesh
            self.stego_mesh = stego_mesh

            return True

        except Exception as e:
            print(f"Error loading models: {str(e)}")
            return False

    def calculate_vertex_error_map(self):
        """Calculate and visualize vertex-wise errors"""
        errors = np.linalg.norm(self.original_vertices - self.stego_vertices, axis=1)
        return errors



    def calculate_psnr(self):
        """Calculate Peak Signal-to-Noise Ratio"""
        mse = np.mean((self.original_vertices - self.stego_vertices) ** 2)
        if mse == 0:
            return float('inf')

        max_val = np.max(self.original_vertices) - np.min(self.original_vertices)
        psnr = 20 * math.log10(max_val / math.sqrt(mse))
        self.metrics['PSNR'] = psnr
        return psnr

    def calculate_ssim(self):
        """Calculate Structural Similarity Index"""
        orig_reshaped = self.original_vertices.reshape(-1, 3)
        stego_reshaped = self.stego_vertices.reshape(-1, 3)

        ssim_scores = []
        for i in range(3):
            score = structural_similarity(
                orig_reshaped[:, i],
                stego_reshaped[:, i],
                data_range=np.max(orig_reshaped[:, i]) - np.min(orig_reshaped[:, i])
            )
            ssim_scores.append(score)

        ssim = np.mean(ssim_scores)
        self.metrics['SSIM'] = ssim
        return ssim

    def calculate_mse(self):
        """Calculate Mean Squared Error"""
        mse = mean_squared_error(self.original_vertices, self.stego_vertices)
        self.metrics['MSE'] = mse
        return mse

    def calculate_rmse(self):
        """Calculate Root Mean Squared Error"""
        rmse = np.sqrt(self.calculate_mse())
        self.metrics['RMSE'] = rmse
        return rmse

    def calculate_ber(self, threshold=1e-6):
        """Calculate Bit Error Rate"""
        differences = np.abs(self.original_vertices - self.stego_vertices)
        binary_orig = (self.original_vertices > threshold).astype(int)
        binary_stego = (self.stego_vertices > threshold).astype(int)

        total_bits = np.prod(binary_orig.shape)
        error_bits = np.sum(binary_orig != binary_stego)

        ber = error_bits / total_bits
        self.metrics['BER'] = ber
        return ber

    def calculate_hausdorff_distance(self):
        """Calculate Hausdorff distance between original and stego models"""
        def directed_hausdorff(source, target):
            tree = KDTree(target)
            distances, _ = tree.query(source)
            return np.max(distances)

        forward = directed_hausdorff(self.original_vertices, self.stego_vertices)
        backward = directed_hausdorff(self.stego_vertices, self.original_vertices)

        hausdorff = max(forward, backward)
        self.metrics['Hausdorff'] = hausdorff
        return hausdorff

    def calculate_histogram_similarity(self, bins=50):
        """Calculate histogram similarity using Wasserstein distance"""
        distances = []

        for i in range(3):
            hist_orig, _ = np.histogram(self.original_vertices[:, i], bins=bins, density=True)
            hist_stego, _ = np.histogram(self.stego_vertices[:, i], bins=bins, density=True)

            distance = wasserstein_distance(hist_orig, hist_stego)
            distances.append(distance)

        hist_similarity = 1 / (1 + np.mean(distances))
        self.metrics['Histogram_Similarity'] = hist_similarity
        return hist_similarity

    def plot_histograms(self, save_path=None):
        """Plot histograms of original and stego models"""
        fig, axes = plt.subplots(1, 3, figsize=(15, 5))
        coords = ['X', 'Y', 'Z']

        for i, (ax, coord) in enumerate(zip(axes, coords)):
            ax.hist(self.original_vertices[:, i], bins=50, alpha=0.5, label='Original', density=True)
            ax.hist(self.stego_vertices[:, i], bins=50, alpha=0.5, label='Stego', density=True)
            ax.set_title(f'{coord} Coordinate Distribution')
            ax.set_xlabel(f'{coord} Value')
            ax.set_ylabel('Density')
            ax.legend()

        plt.tight_layout()
        if save_path:
            plt.savefig(save_path)
            plt.close()
        else:
            plt.show()

    def plot_error_distribution(self, save_path=None):
        """Plot the distribution of geometric errors"""
        errors = np.linalg.norm(self.original_vertices - self.stego_vertices, axis=1)

        plt.figure(figsize=(10, 6))
        plt.hist(errors, bins=50, density=True)
        plt.title('Geometric Error Distribution')
        plt.xlabel('Error Magnitude')
        plt.ylabel('Density')

        if save_path:
            plt.savefig(save_path)
            plt.close()
        else:
            plt.show()

    def evaluate_all(self, plot=True, save_plots=False):
        """Calculate all metrics and optionally generate plots"""
        metrics = {
            'PSNR': self.calculate_psnr(),
            'SSIM': self.calculate_ssim(),
            'MSE': self.calculate_mse(),
            'RMSE': self.calculate_rmse(),
            'BER': self.calculate_ber(),
            'Hausdorff': self.calculate_hausdorff_distance(),
            'Histogram_Similarity': self.calculate_histogram_similarity()
        }

        if plot:
            self.plot_histograms(save_path='histograms.png' if save_plots else None)
            self.plot_error_distribution(save_path='error_distribution.png' if save_plots else None)
            #self.visualize_error_map(save_path='error_map.png' if save_plots else None)

        return metrics
    def generate_report(self, output_path='/content/drive/MyDrive/OutputPaper2/evaluation_report.txt'):
        """Generate a detailed evaluation report"""
        with open(output_path, 'w') as f:
            f.write("3D Steganography Evaluation Report\n")
            f.write("=================================\n\n")

            # Model information
            f.write("Model Information:\n")
            f.write(f"Number of vertices: {len(self.original_vertices)}\n")
            f.write(f"Number of faces: {len(self.original_mesh.faces)}\n\n")

            # Metrics
            f.write("Quality Metrics:\n")
            for metric, value in self.metrics.items():
                f.write(f"{metric}: {value:.6f}\n")

            f.write("\nInterpretation:\n")
            f.write("- PSNR > 30 dB typically indicates good quality\n")
            f.write("- SSIM closer to 1 indicates better structural preservation\n")
            f.write("- Lower MSE and RMSE values indicate better similarity\n")
            f.write("- Lower BER indicates better steganographic accuracy\n")
            f.write("- Histogram similarity closer to 1 indicates better statistical imperceptibility\n")

def main():
    # Initialize evaluator
    evaluator = Stego3DEvaluator()

    # Load and align models
    if not evaluator.load_models('/content/drive/MyDrive/OutputPaper2/modified_pjanic.obj', '/content/drive/MyDrive/OutputPaper2/pjanic.obj', align=True):
        return

    # Perform evaluation
    metrics = evaluator.evaluate_all(plot=True, save_plots=True)

    # Print results
    print("\nEvaluation Results:")
    for metric, value in metrics.items():
        print(f"{metric}: {value:.6f}")
    evaluator.generate_report()
    print("\nDetailed report saved to '/content/drive/MyDrive/OutputPaper2/evaluation_report.txt'")

if __name__ == "__main__":
    main()

Original vertices: 215999
Stego vertices: 220141

Alignment Statistics:
Mean distance: 0.000911
Max distance: 0.000959
Aligned vertices: 220141

Evaluation Results:
PSNR: 69.809654
SSIM: 0.999811
MSE: 0.000000
RMSE: 0.000526
BER: 0.000486
Hausdorff: 0.000959
Histogram_Similarity: 0.999544

Detailed report saved to '/content/drive/MyDrive/OutputPaper2/evaluation_report.txt'
